[Reference](https://nandakishorej8.medium.com/multivariate-timeseries-forecast-with-lead-and-lag-timesteps-using-lstm-1a34915f08a)

In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_lag=1, n_lead=1, dropnan=True):
 n_vars = 1 if type(data) is list else data.shape[1]
 df = DataFrame(data)
 cols, names = list(), list()
 # input sequence (t-n, … t-1)
 for i in range(n_lag, 0, -1):
 cols.append(df.shift(i))
 names += [(‘var%d(t-%d)’ % (j+1, i)) for j in range(n_vars)]
 # forecast sequence (t, t+1, … t+n)
 for i in range(0, n_lead):
 cols.append(df.shift(-i))
 if i == 0:
 names += [(‘var%d(t)’ % (j+1)) for j in range(n_vars)]
 else:
 names += [(‘var%d(t+%d)’ % (j+1, i)) for j in range(n_vars)]
 # put it all together
 agg = concat(cols, axis=1)
 agg.columns = names
 # drop rows with NaN values
 if dropnan:
 agg.dropna(inplace=True)
 return agg

reframed = series_to_supervised(values, n_lag, (n_lead+1))
#removing the future (t+n) dependent variable (Y)
if n_lead>0:
  reframed= reframed.drop(reframed.iloc[:,[i for i in range(df_no.shape[1]*(n_lag+1),reframed.shape[1],df_no.shape[1])]],axis=1)

In [1]:
# splitting reframed to X and Y considering the first column to be out target feature
X=reframed.drop(['var1(t)'],axis=1)
Y=reframed['var1(t)']
X_values=X.values
Y_values=Y.values
#n_preduct being the test length
train_X,train_Y = X_values[:(X_values.shape[0]-n_predict),:],Y_values[:(X_values.shape[0]-n_predict)]
test_X,test_Y = X_values[(X_values.shape[0]-n_predict):,:],Y_values[(X_values.shape[0]-n_predict):]
#reshaping train and test to feed to LSTM
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01)
model = Sequential()
model.add(LSTM(100,return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.25))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.20))
model.add(LSTM(units=10,return_sequences=False))
model.add(Dense(units=1, activation='linear'))
model.compile(loss='mae', optimizer=opt)

#adding few model check points
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
  rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=10, verbose=1)
  mcp = ModelCheckpoint(filepath="/test.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)
  tb = TensorBoard('logs')
history = model.fit(train_X, train_Y, epochs=50, batch_size=10,callbacks=[mcp,rlr],validation_data=(test_X, test_Y), verbose=2, shuffle=False)

yhat = model.predict(test_X)